In [1]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50V2  # Using ResNet50V2
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split


In [2]:

#  1. Define the Pre-trained ResNet50V2 Model
base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False







94668760/94668760 ━━━━━━━━━━━━━━━━━━━━ 12s 0us/step


In [3]:
# 2. Add New Layers on Top of VGG16
# 2. Add New Layers on Top of ResNet50V2
x = Flatten()(base_model.output)
x = Dense(1024, activation='relu')(x)

predictions = Dense(2, activation='softmax')(x)


In [4]:

# 3. Create the Model
model = Model(inputs=base_model.input, outputs=predictions)

In [5]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=[
        'accuracy',              # Basic accuracy metric
    #     tf.keras.metrics.Precision(name='precision'),  # Precision metric
    #     tf.keras.metrics.Recall(name='recall'),        # Recall metric
    #     tf.keras.metrics.AUC(name='auc')               # AUC (Area Under the ROC Curve)
     ]
)

In [6]:
# 3. Data Augmentation and Loading

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

datagen_no_aug = ImageDataGenerator(rescale=1./255)

all_images = datagen_no_aug.flow_from_directory(
    r'dataset',
    target_size=(224, 224),
    batch_size=663,  # Load all images at once (adjust if needed)
    class_mode='binary',
    shuffle=False
)


Found 593 images belonging to 2 classes.


In [7]:


# 4. Split the Dataset
X_all, y_all = next(all_images)
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# 5. Create Data Generators for Each Set
training_set = train_datagen.flow(X_train, y_train, batch_size=32)
validation_set = datagen_no_aug.flow(X_val, y_val, batch_size=32)
test_set = datagen_no_aug.flow(X_test, y_test, batch_size=32, shuffle=False)


In [8]:

# 6. Train the Model
history = model.fit(
    training_set,
    epochs=25,  # Adjust the number of epochs as needed
    validation_data=validation_set
)




Epoch 1/25


c:\Users\Manjil\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


12/12 ━━━━━━━━━━━━━━━━━━━━ 74s 5s/step - accuracy: 0.4941 - loss: 7.8309 - val_accuracy: 0.3782 - val_loss: 0.9876
Epoch 2/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 45s 3s/step - accuracy: 0.4622 - loss: 1.2019 - val_accuracy: 0.8487 - val_loss: 0.5359
Epoch 3/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 42s 3s/step - accuracy: 0.7348 - loss: 0.5513 - val_accuracy: 0.6639 - val_loss: 0.5848
Epoch 4/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 48s 4s/step - accuracy: 0.7272 - loss: 0.5523 - val_accuracy: 0.6807 - val_loss: 0.5023
Epoch 5/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 42s 3s/step - accuracy: 0.7561 - loss: 0.5115 - val_accuracy: 0.8824 - val_loss: 0.4235
Epoch 6/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 42s 3s/step - accuracy: 0.8423 - loss: 0.4156 - val_accuracy: 0.9244 - val_loss: 0.3638
Epoch 7/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 41s 3s/step - accuracy: 0.7963 - loss: 0.4497 - val_accuracy: 0.8655 - val_loss: 0.3423
Epoch 8/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 37s 3s/step - accuracy: 0.8606 - loss: 0.3825 - val_accuracy: 0.9076 - val_loss: 0.3242
Epo

In [9]:
# 7. Save the Trained Model


# Assume you have a trained model named `model`
model.save('liveliness_detection_model_resnet.h5')





In [10]:

loss, accuracy = model.evaluate(test_set)   # Use the test_set generator directly
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.9179 - loss: 0.2183
Test Loss: 0.20079153776168823
Test Accuracy: 0.9327731132507324
